In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced

In [2]:
# Import the BITCOIN csv into the notebook
signals_df = pd.read_csv(
    Path("./Resources/signals_df.csv"),
    index_col = 'Date',
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
signals_df.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159,-1.0,0.0,1.0,0,NaN,3.827184,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-1.0,0.0,-1.0,0,0.0,-3.575799,-2.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-1.0,0.0,-1.0,0,0.0,-4.006930,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-1.0,0.0,-1.0,0,0.0,-3.870433,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-1.0,0.0,-1.0,0,0.0,-0.903208,0.0


In [3]:
# Create a list of categorical variables 
#categorical_variables = list(stoch_df.dtypes[stoch_df.dtypes == "object"].index)
categorical_variables = list(signals_df[['MACD_Signal','RSI_Signal','Stoch_Signal','custom_signal','MACD_Entry/Exit','Stoch_Entry/Exit']])
# Display the categorical variables list
display(categorical_variables[0:6])

['MACD_Signal',
 'RSI_Signal',
 'Stoch_Signal',
 'custom_signal',
 'MACD_Entry/Exit',
 'Stoch_Entry/Exit']

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
    # sparse = False, results in an array
    # sparse = True (default), results in a sparse matrix

In [5]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(signals_df[categorical_variables])
encoded_data[0:1]

array([[1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 1.]])

In [6]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
        # function gathers column names and assigns them to the new DataFrame
)

# set index of encoded_df
encoded_df.set_index(signals_df.index, inplace=True)

# Review the DataFrame
encoded_df.head()

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,MACD_Signal_-1.0,MACD_Signal_1.0,RSI_Signal_-1.0,RSI_Signal_0.0,RSI_Signal_1.0,Stoch_Signal_-1.0,Stoch_Signal_1.0,custom_signal_-2,custom_signal_0,custom_signal_2,MACD_Entry/Exit_-2.0,MACD_Entry/Exit_0.0,MACD_Entry/Exit_2.0,MACD_Entry/Exit_nan,Stoch_Entry/Exit_-2.0,Stoch_Entry/Exit_0.0,Stoch_Entry/Exit_2.0,Stoch_Entry/Exit_nan
Date,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2021-04-20 09:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2021-04-20 10:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2021-04-20 11:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2021-04-20 12:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
encoded_df.drop(columns=['Stoch_Entry/Exit_nan','MACD_Entry/Exit_nan'],inplace=True)

# Review the DataFrame
encoded_df.head()

,MACD_Signal_-1.0,MACD_Signal_1.0,RSI_Signal_-1.0,RSI_Signal_0.0,RSI_Signal_1.0,Stoch_Signal_-1.0,Stoch_Signal_1.0,custom_signal_-2,custom_signal_0,custom_signal_2,MACD_Entry/Exit_-2.0,MACD_Entry/Exit_0.0,MACD_Entry/Exit_2.0,Stoch_Entry/Exit_-2.0,Stoch_Entry/Exit_0.0,Stoch_Entry/Exit_2.0
Date,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-04-20 09:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2021-04-20 10:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2021-04-20 11:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2021-04-20 12:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [8]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
side_numeric = signals_df.drop(columns=['MACD_Signal','RSI_Signal','Stoch_Signal','custom_signal','MACD_Entry/Exit','Stoch_Entry/Exit'])

# Review the DataFrame
side_numeric.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,stoch_diff
Date,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159,3.827184
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-3.575799
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-4.006930
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-3.870433
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-0.903208


In [9]:
signals_ohe_df = pd.concat([encoded_df,side_numeric],axis=1)

# Review the number of columns
len(signals_ohe_df.columns)

28

In [10]:
X = signals_ohe_df.drop(columns=['custom_signal_-2','custom_signal_0','custom_signal_2'])

# Review the number of columns
len(X.columns)

25

In [11]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_ohe_df[['custom_signal_-2','custom_signal_0','custom_signal_2']]

# Review the number of columns
len(y.columns)

3

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    # test_size=x
# Review the DataFrame
X_test.head()

,MACD_Signal_-1.0,MACD_Signal_1.0,RSI_Signal_-1.0,RSI_Signal_0.0,RSI_Signal_1.0,Stoch_Signal_-1.0,Stoch_Signal_1.0,MACD_Entry/Exit_-2.0,MACD_Entry/Exit_0.0,MACD_Entry/Exit_2.0,...,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,stoch_diff
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-19 10:30:00-05:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,456.730011,454.730011,-0.001664,-3.000169,-0.401513,-2.598656,30.691537,10.702383,12.275826,-1.573443
2021-11-01 15:30:00-04:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,460.230011,459.369995,0.001502,0.946894,0.011684,0.935211,62.196640,74.029651,71.058528,2.971122
2022-01-05 15:30:00-05:00,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,471.190002,468.290009,-0.004072,-1.096733,-0.979166,-0.117567,25.288324,2.290720,8.693256,-6.402536
2021-09-15 10:30:00-04:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,445.570007,444.220001,0.001980,-1.253207,0.081520,-1.334727,42.921522,26.847198,16.588462,10.258736
2021-10-28 12:30:00-04:00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,457.429993,456.500000,-0.000547,0.812403,-0.111141,0.923544,57.863902,80.863976,70.114403,10.749573


In [13]:
# Select the start of the training period
training_begin = X.index.min() + DateOffset(hours=1)

# Display the training begin date
print(training_begin)

2021-04-19 16:30:00-04:00


In [14]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)
    # Keep training less than 50% of total DataFrame

# Display the training end date
print(training_end)

2021-07-19 15:30:00-04:00


In [15]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,MACD_Signal_-1.0,MACD_Signal_1.0,RSI_Signal_-1.0,RSI_Signal_0.0,RSI_Signal_1.0,Stoch_Signal_-1.0,Stoch_Signal_1.0,MACD_Entry/Exit_-2.0,MACD_Entry/Exit_0.0,MACD_Entry/Exit_2.0,...,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,stoch_diff
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-20 09:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-3.575799
2021-04-20 10:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-4.006930
2021-04-20 11:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-3.870433
2021-04-20 12:30:00-04:00,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-0.903208
2021-04-20 13:30:00-04:00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,412.100006,411.200012,0.000596,-0.422760,-0.740461,0.317701,35.707715,14.220449,12.578948,1.641501


In [16]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test.head()

,MACD_Signal_-1.0,MACD_Signal_1.0,RSI_Signal_-1.0,RSI_Signal_0.0,RSI_Signal_1.0,Stoch_Signal_-1.0,Stoch_Signal_1.0,MACD_Entry/Exit_-2.0,MACD_Entry/Exit_0.0,MACD_Entry/Exit_2.0,...,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,stoch_diff
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-20 09:30:00-04:00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,429.470001,424.829987,0.009554,-2.727512,-0.308362,-2.419150,47.721606,27.075173,14.110055,12.965119
2021-07-20 10:30:00-04:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,430.559998,428.809998,0.002098,-2.288172,0.104782,-2.392954,50.323379,47.861584,28.632651,19.228933
2021-07-20 11:30:00-04:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,431.440002,429.839996,0.003024,-1.814182,0.463018,-2.277200,53.892707,66.275627,47.070795,19.204832
2021-07-20 12:30:00-04:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,432.079987,430.839996,-0.000603,-1.442889,0.667449,-2.110338,53.071355,72.896502,62.344571,10.551931
2021-07-20 13:30:00-04:00,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,431.829987,430.739990,0.001949,-1.068539,0.833439,-1.901978,55.434399,81.425578,73.532569,7.893009


In [17]:
# Scale the features DataFrames
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

## Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# MACD Neural Network
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

25

In [19]:
# Define the number of neurons in the output layer
number_output_neurons = 3
    # equal to one because we only have one target, y.
    # should be equal to the number of target columns we are trying to predict.

In [20]:
n = '\n'

# Define the number of hidden nodes for the first hidden layer and second layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) //2
hidden_nodes_layer2 = (hidden_nodes_layer1 + number_output_neurons) //2
hidden_nodes_layer3 = (hidden_nodes_layer2 + number_output_neurons) //2 
hidden_nodes_layer4 = (hidden_nodes_layer3 + number_output_neurons) //2 
hidden_nodes_layer5 = (hidden_nodes_layer4 + number_output_neurons) //2 

# Review the number hidden nodes in the first and second layer
print(f'# of neurons in the first hidden layer: {hidden_nodes_layer1}{n}# of neurons in the second hidden layer: {hidden_nodes_layer2}{n}# of neurons in the third hidden layer: {hidden_nodes_layer3}'
    f'{n}# of neurons in the fourth hidden layer: {hidden_nodes_layer4}{n}# of neurons in the fifth hidden layer: {hidden_nodes_layer5}{n}')

# of neurons in the first hidden layer: 14
# of neurons in the second hidden layer: 8
# of neurons in the third hidden layer: 5
# of neurons in the fourth hidden layer: 4
# of neurons in the fifth hidden layer: 3



In [21]:
# Create the Sequential model instance
nn = Sequential()

In [22]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

In [23]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

In [24]:
# Add the third hidden layer
nn.add(Dense(units=hidden_nodes_layer3,activation='relu'))

In [25]:
# Add the fourth hidden layer
#nn.add(Dense(units=hidden_nodes_layer4,activation='relu'))

In [26]:
# Add the fifth hidden layer
#nn.add(Dense(units=hidden_nodes_layer5,activation='relu'))

In [27]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons,activation='softmax'))

In [28]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                364       
                                                                 
 dense_1 (Dense)             (None, 8)                 120       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                                 
 dense_3 (Dense)             (None, 3)                 18        
                                                                 
Total params: 547
Trainable params: 547
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the Sequential model
nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

    # metric exploration

In [30]:
# Fit the model using 50 epochs and the training data
nn.fit(X_train_scaled,y_train,epochs=100, verbose=3)
    # make sure to use X_train_scaled rather than X_train
    # verbose=3, reduces the graphics displayed per epoch. in turn this increases the overall speed of the epochs.

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [31]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
nn_signals_loss, nn_signals_accuracy, nn_signals_mse = nn.evaluate(X_test_scaled,y_test, verbose=3)

# Display the model loss and accuracy results
abc = print(f"Loss: {nn_signals_loss}, Accuracy: {nn_signals_accuracy}, MSE: {nn_signals_mse}")

Loss: 0.09457379579544067, Accuracy: 0.9861538410186768, MSE: 0.008681447245180607


In [32]:
# Loss: 0.2134855091571808, Accuracy: 0.9853846430778503
# # of neurons in the first hidden layer: 14
# of neurons in the second hidden layer: 8
# of neurons in the third hidden layer: 5
# of neurons in the fourth hidden layer: 4
# of neurons in the fifth hidden layer: 3
# relu activations functions with softmax for the output layer
# loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy, mse']
# 3 output neurons

In [33]:
signals_df.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159,-1.0,0.0,1.0,0,NaN,3.827184,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-1.0,0.0,-1.0,0,0.0,-3.575799,-2.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-1.0,0.0,-1.0,0,0.0,-4.006930,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-1.0,0.0,-1.0,0,0.0,-3.870433,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-1.0,0.0,-1.0,0,0.0,-0.903208,0.0


In [34]:
signals_df.dtypes

Close               float64
Open                float64
High                float64
Low                 float64
actual_returns      float64
MACD_12_26_9        float64
MACDh_12_26_9       float64
MACDs_12_26_9       float64
RSI_14              float64
STOCHk_14_3_3       float64
STOCHd_14_3_3       float64
MACD_Signal         float64
RSI_Signal          float64
Stoch_Signal        float64
custom_signal         int64
MACD_Entry/Exit     float64
stoch_diff          float64
Stoch_Entry/Exit    float64
dtype: object

In [35]:
categorical_variables = list(signals_df[['custom_signal']])
categorical_variables

['custom_signal']

In [36]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(signals_df[categorical_variables])
encoded_data[0:5]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [37]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
        # function gathers column names and assigns them to the new DataFrame
)

# set index of encoded_df
encoded_df.set_index(signals_df.index, inplace=True)

# Review the DataFrame
encoded_df.head()

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,custom_signal_-2,custom_signal_0,custom_signal_2
Date,,,
2021-04-19 15:30:00-04:00,0.0,1.0,0.0
2021-04-20 09:30:00-04:00,0.0,1.0,0.0
2021-04-20 10:30:00-04:00,0.0,1.0,0.0
2021-04-20 11:30:00-04:00,0.0,1.0,0.0
2021-04-20 12:30:00-04:00,0.0,1.0,0.0


In [38]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
side_numeric = signals_df.drop(columns=['custom_signal'])
side_numeric.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159,-1.0,0.0,1.0,NaN,3.827184,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-1.0,0.0,-1.0,0.0,-3.575799,-2.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-1.0,0.0,-1.0,0.0,-4.006930,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-1.0,0.0,-1.0,0.0,-3.870433,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-1.0,0.0,-1.0,0.0,-0.903208,0.0


In [39]:
signals_df = pd.concat([encoded_df,side_numeric],axis=1)
signals_df.head()

,custom_signal_-2,custom_signal_0,custom_signal_2,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,0.0,1.0,0.0,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159,-1.0,0.0,1.0,NaN,3.827184,NaN
2021-04-20 09:30:00-04:00,0.0,1.0,0.0,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-1.0,0.0,-1.0,0.0,-3.575799,-2.0
2021-04-20 10:30:00-04:00,0.0,1.0,0.0,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-1.0,0.0,-1.0,0.0,-4.006930,0.0
2021-04-20 11:30:00-04:00,0.0,1.0,0.0,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-1.0,0.0,-1.0,0.0,-3.870433,0.0
2021-04-20 12:30:00-04:00,0.0,1.0,0.0,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-1.0,0.0,-1.0,0.0,-0.903208,0.0


In [40]:
X = signals_df.drop(columns=['custom_signal_-2','custom_signal_0','custom_signal_2'])

X.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,
2021-04-19 15:30:00-04:00,415.239990,414.325012,415.339996,414.140015,0.002208,0.647863,-0.549211,1.197073,52.330591,25.930343,22.103159,-1.0,0.0,1.0,NaN,3.827184,NaN
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-1.0,0.0,-1.0,0.0,-3.575799,-2.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-1.0,0.0,-1.0,0.0,-4.006930,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-1.0,0.0,-1.0,0.0,-3.870433,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-1.0,0.0,-1.0,0.0,-0.903208,0.0


In [41]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Stoch_Entry/Exit']

y

Date
2021-04-19 15:30:00-04:00    NaN
2021-04-20 09:30:00-04:00   -2.0
2021-04-20 10:30:00-04:00    0.0
2021-04-20 11:30:00-04:00    0.0
2021-04-20 12:30:00-04:00    0.0
                            ... 
2022-04-12 12:30:00-04:00   -2.0
2022-04-12 13:30:00-04:00    0.0
2022-04-12 14:30:00-04:00    0.0
2022-04-12 15:30:00-04:00    0.0
2022-04-12 16:00:00-04:00    2.0
Name: Stoch_Entry/Exit, Length: 1742, dtype: float64

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    # test_size=x
len(X_test)

436

In [43]:
y_train.value_counts()

 0.0    1006
 2.0     151
-2.0     149
Name: Stoch_Entry/Exit, dtype: int64

In [44]:
# Select the start of the training period
training_begin = X.index.min() + DateOffset(hours=1)

# Display the training begin date
print(training_begin)

2021-04-19 16:30:00-04:00


In [45]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)
    # Keep training less than 50% of total DataFrame

# Display the training end date
print(training_end)

2021-07-19 15:30:00-04:00


In [46]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,
2021-04-20 09:30:00-04:00,413.920013,413.910004,414.679993,413.660004,-0.003179,0.487409,-0.567732,1.055140,43.742207,18.099063,21.674862,-1.0,0.0,-1.0,0.0,-3.575799,-2.0
2021-04-20 10:30:00-04:00,411.575012,413.920990,413.929993,411.119995,-0.005665,0.169077,-0.708851,0.877928,33.289577,16.004308,20.011238,-1.0,0.0,-1.0,0.0,-4.006930,0.0
2021-04-20 11:30:00-04:00,412.144989,411.575012,412.399994,410.619995,0.001385,-0.036787,-0.731771,0.694985,37.216643,11.246037,15.116469,-1.0,0.0,-1.0,0.0,-3.870433,0.0
2021-04-20 12:30:00-04:00,411.290009,412.149994,412.220001,411.250000,-0.002074,-0.265860,-0.768676,0.502816,33.984850,12.270360,13.173568,-1.0,0.0,-1.0,0.0,-0.903208,0.0
2021-04-20 13:30:00-04:00,411.535004,411.309692,412.100006,411.200012,0.000596,-0.422760,-0.740461,0.317701,35.707715,14.220449,12.578948,-1.0,0.0,1.0,0.0,1.641501,2.0


In [47]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())
    # NOT SURE IF DATEOFFSET IS NECESSARY FOR X/Y TEST

,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,
2021-07-20 09:30:00-04:00,429.019989,425.679993,429.470001,424.829987,0.009554,-2.727512,-0.308362,-2.419150,47.721606,27.075173,14.110055,-1.0,0.0,1.0,0.0,12.965119,0.0
2021-07-20 10:30:00-04:00,429.920013,429.019989,430.559998,428.809998,0.002098,-2.288172,0.104782,-2.392954,50.323379,47.861584,28.632651,1.0,0.0,1.0,2.0,19.228933,0.0
2021-07-20 11:30:00-04:00,431.220001,429.920013,431.440002,429.839996,0.003024,-1.814182,0.463018,-2.277200,53.892707,66.275627,47.070795,1.0,0.0,1.0,0.0,19.204832,0.0
2021-07-20 12:30:00-04:00,430.959991,431.220001,432.079987,430.839996,-0.000603,-1.442889,0.667449,-2.110338,53.071355,72.896502,62.344571,1.0,0.0,1.0,0.0,10.551931,0.0
2021-07-20 13:30:00-04:00,431.799988,430.959991,431.829987,430.739990,0.001949,-1.068539,0.833439,-1.901978,55.434399,81.425578,73.532569,1.0,0.0,1.0,0.0,7.893009,0.0


,Close,Open,High,Low,actual_returns,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,
2022-04-12 12:30:00-04:00,441.640015,441.750000,442.700012,441.170013,-0.000340,-1.904712,-0.045560,-1.859152,39.286301,28.591089,30.039237,-1.0,0.0,-1.0,0.0,-1.448148,-2.0
2022-04-12 13:30:00-04:00,438.489990,441.649994,441.940002,438.420013,-0.007133,-2.125682,-0.213224,-1.912458,32.802059,14.954800,25.692469,-1.0,0.0,-1.0,0.0,-10.737669,0.0
2022-04-12 14:30:00-04:00,437.059998,438.480011,439.290009,436.650085,-0.003261,-2.388655,-0.380958,-2.007697,30.352869,8.822045,17.455978,-1.0,0.0,-1.0,0.0,-8.633933,0.0
2022-04-12 15:30:00-04:00,438.269989,437.059906,438.779999,436.679993,0.002768,-2.470944,-0.370598,-2.100347,34.789679,7.240768,10.339204,-1.0,0.0,-1.0,0.0,-3.098436,0.0
2022-04-12 16:00:00-04:00,438.290009,438.290009,438.290009,438.290009,0.000046,-2.505660,-0.324251,-2.181409,34.863614,13.034849,9.699221,-1.0,0.0,1.0,0.0,3.335629,2.0


In [48]:
# Scale the features DataFrames
# Create a StandardScaler instance

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

## Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [49]:
# From SVM, instantiate SVC classifier model instance
signals_tree = RandomForestClassifier(random_state=1)
 
# Fit the model to the data using the training data
signals_tree.fit(X_train_scaled,y_train)
 
# Use the testing data to make the model predictions
y_signals_tree_pred = signals_tree.predict(X_test_scaled)

signals_tree_class = classification_report(y_test,y_signals_tree_pred)
signals_tree_matrix = confusion_matrix(y_test,y_signals_tree_pred)

In [50]:
# Instantiate the random oversampler model

random_sampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = random_sampler.fit_resample(X_train,y_train)


y_resampled.value_counts()

# Do we have to create this before 

-2.0    327
 0.0    327
 2.0    327
Name: Stoch_Entry/Exit, dtype: int64

In [51]:
# Stoch RandomForestClassifier Oversampled
signals_tree_os = RandomForestClassifier(random_state=1)
 
# Fit the model to the data using the training data
signals_tree_os.fit(X_resampled,y_resampled)
 
# Use the testing data to make the model predictions
y_signals_tree_pred_os = signals_tree_os.predict(X_test)

signals_tree_class_os = classification_report(y_test,y_signals_tree_pred_os)
signals_tree_matrix_os = confusion_matrix(y_test,y_signals_tree_pred_os)

In [52]:
# # From LogisticRegression, instantiate LogisticRegression classifier model instance
signals_log = LogisticRegression(random_state=1)
 
 # Fit the model to the data using the training data
signals_log.fit(X_train_scaled,y_train)
 
# # Use the testing data to make the model predictions
y_signals_log_pred = signals_log.predict(X_test_scaled)

# # Create and save confusion matrix and classification report to a variable name
signals_log_matrix = confusion_matrix(y_test,y_signals_log_pred)
signals_log_class = classification_report(y_test,y_signals_log_pred)

In [53]:
print(signals_log_matrix)
print(signals_log_class)

[[ 150    0    0]
 [   0 1000    0]
 [   0    0  150]]
              precision    recall  f1-score   support

        -2.0       1.00      1.00      1.00       150
         0.0       1.00      1.00      1.00      1000
         2.0       1.00      1.00      1.00       150

    accuracy                           1.00      1300
   macro avg       1.00      1.00      1.00      1300
weighted avg       1.00      1.00      1.00      1300



In [54]:
# From LogisticRegression, instantiate LogisticRegression classifier model instance
signals_log_os = LogisticRegression(random_state=1)
 
# Fit the model to the data using the training data
signals_log_os.fit(X_resampled,y_resampled)
 
# # Use the testing data to make the model predictions
y_signals_log_pred_os = signals_log.predict(X_test)

# Create and save confusion matrix and classification report to a variable name
signals_log_matrix_os = confusion_matrix(y_test,y_signals_log_pred_os)
signals_log_class_os = classification_report(y_test,y_signals_log_pred_os)

C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\base.py:439: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no p

In [55]:
# NOW TRY TO TEST

In [56]:
print('NN')
print(f'Loss: {nn_signals_loss}, Accuracy: {nn_signals_accuracy}, MSE: {nn_signals_mse}')
print('Random Forest')
print(signals_tree_class)
print(signals_tree_matrix)
print('OS -- Random Forest')
print(signals_tree_class_os)
print(signals_tree_matrix_os)
print('log Reg')
print(signals_log_matrix_os)
print(signals_log_class_os)
print('OS -- Log Reg')
print(signals_log_matrix_os)
print(signals_log_class_os)

NN
Loss: 0.09457379579544067, Accuracy: 0.9861538410186768, MSE: 0.008681447245180607
Random Forest
              precision    recall  f1-score   support

        -2.0       1.00      1.00      1.00       150
         0.0       1.00      1.00      1.00      1000
         2.0       1.00      1.00      1.00       150

    accuracy                           1.00      1300
   macro avg       1.00      1.00      1.00      1300
weighted avg       1.00      1.00      1.00      1300

[[ 150    0    0]
 [   0 1000    0]
 [   0    0  150]]
OS -- Random Forest
              precision    recall  f1-score   support

        -2.0       1.00      1.00      1.00       150
         0.0       1.00      1.00      1.00      1000
         2.0       1.00      1.00      1.00       150

    accuracy                           1.00      1300
   macro avg       1.00      1.00      1.00      1300
weighted avg       1.00      1.00      1.00      1300

[[ 150    0    0]
 [   0 1000    0]
 [   0    0  150]]
log Reg


In [57]:
# Fit the model using 50 epochs and the training data
nn.fit(X_test_scaled,y_test,epochs=100, verbose=3)

Epoch 1/100


ValueError: in user code:

    File "C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\kjg20\anaconda3\envs\dev\envs\dev\lib\site-packages\keras\engine\

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
nn_signals_loss, nn_signals_accuracy = nn.evaluate(X_test_scaled,y_test, verbose=3)

# Display the model loss and accuracy results
abc = print(f"Loss: {nn_signals_loss}, Accuracy: {nn_signals_accuracy}")